# Hauptkomponentenanalyse - Principal Component Analysis (PCA)

## Introduction
Das Ziel der PCA ist, Muster in Daten zu identifizieren. Die PCA zielt darauf ab, die Korrelation zwischen Variablen zu ermitteln. Die Intention, die Dimensionalität zu reduzieren, ist nur dann sinnvoll, wenn eine starke Korrelation zwischen Variablen besteht.

Kurz gesagt, bei der PCA geht es um folgendes: Ermittlung der Richtungen mit der maximalen Varianz in hochdimensionalen Daten und eine anschließende Projektion auf einen kleineren dimensionalen Unterraum, während die meisten Informationen behalten werden.

## PCA and Dimensionsreduktion
Häufig besteht das gewünschte Ziel darin, die Dimensionen eines d-dimensionalen Datensatzes zu reduzieren, indem er auf einen (k)-dimensionalen Unterraum projiziert wird (wobei k <d ist), um die Recheneffizienz zu erhöhen und gleichzeitig den größten Teil der Informationen beizubehalten.

## Eine Zusammenfassung des PCA-Ansatzes
* Ermittlung der Eigenvektoren (Eigenvectors) und Eigenwerte (Eigenvalues) anhand Kovarianzmatrix <br>
* Sortierung der Eigenwerte in absteigender Reihenfolge und Auswahl der k Eigenvektoren, die den k größten Eigenwerten entsprechen, wobei k die Anzahl der Dimensionen des neuen Merkmalsunterraums ist (k <= d).
* Konstruktion der Projektionsmatrix <b>W</b> aus den ausgewählten k Eigenvektoren.
* Transformierung des ursprünglichen Datensatzes <b>X </b> über <b>W</b>, um einen k-dimensionalen Merkmalsunterraum <b>Y</b> zu erhalten.

## Iris Datensatz

Für die anstehende Aufgabe wird der bereits bekannte Iris-Datensatz verwendet.

<img src="./Figures/Iris-Datensammlung.png" alt="drawing" style="width:400px;"/>

## Datensatz einlesen

Lesen Sie den Iris-Datensatz aus dem Verzeichnis /Data ein. Speichern Sie hierbei die numerischen Werte und die Class labels in unterschiedlichen Variablen.

In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

In [2]:
# TODO: implement
file = "./Data/iris.data"
df = pd.read_csv(file, names= ["SepalLength", "SepalWidth" , "PetalLength", "Petal Width","Target"])
df.reset_index()
X= np.array(df.loc[:, ["SepalLength", "SepalWidth" , "PetalLength", "Petal Width"]]).reshape(-1, 4)
y= np.array(df.loc[:,"Target"]).reshape(-1, 1)

In [3]:
print(X[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[['Iris-setosa']
 ['Iris-setosa']
 ['Iris-setosa']
 ['Iris-setosa']
 ['Iris-setosa']]


Der Iris-Datensatz ist jetzt in Form einer 150$\times$4-Matrix gespeichert, in der die Spalten die verschiedenen Merkmale darstellen und jede Zeile eine separate Blumenprobe darstellt. Jedes Sample <b>x</b> kann als 4-dimensionaler Vektor dargestellt werden

In [4]:
X.shape

(150, 4)

In [5]:
X[0:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

## Visualisierung

Die Daten (3 verschiedene Blumenklassen befinden sich entlang der 4 verschiedenen Merkmale) lassen sich über Histogramme wiefolgt visualisieren.

Figure:
<img src="./Figures/Exploratory-Visualization.png" alt="drawing" style="width:400px;"/>

# 1 - Berechnung der Kovarianzmatrix

Die Eigenvektoren und Eigenwerte einer Kovarianzmatrix bilden den „Kern“ einer PCA: Die Eigenvektoren (Hauptkomponenten) bestimmen die Richtungen des neuen Merkmalsraums, und die Eigenwerte bestimmen ihre Größe. Mit anderen Worten, die Eigenwerte erklären die Varianz der Daten entlang der neuen Merkmalsachsen.

## Kovarianzmatrix C
Die Herangehensweise der PCA besteht darin, die Eigendekomposition anhand der Kovarianzmatrix $\Sigma$ vorzunehmen. Diese ist ein d$\times$d-Matrix, bei der jedes Element die Kovarianz zwischen zwei Merkmalen darstellt. Die Kovarianz zwischen zwei Merkmalen wird wie folgt berechnet:

$\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^{n}\left(  x_{ij}-\bar{x}_j \right)  \left( x_{ik}-\bar{x}_k \right).$

Die Berechnung der Kovarianzmatrix wird über die folgende Matrixgleichung zusammengefasst:
$\Sigma = \frac{1}{n-1} \left( (\mathbf{X} - \mathbf{\bar{x}})^T\;(\mathbf{X} - \mathbf{\bar{x}}) \right)$

mit $\bar{x}$ als Mittelwertvektor $\mathbf{\bar{x}} = \frac{1}{n} \sum\limits_{i=1}^n x_{i}.$

Der Mittelwertvektor (mean vector) ist ein d-dimensionaler Vektor, bei dem jeder Wert in diesem Vektor den Stichprobenmittelwert einer Merkmalsspalte im Datensatz darstellt.

Implementieren Sie eigenständig den Mittelwertvektor und die Kovarianzmatrix. Testen Sie das Ergebnis Ihrer Implementierung anhand der Funktion <b>numpy.cov()</b>.

In [6]:
# TODO: implement
test_cov = np.cov(X.T)
print("Test: \n{}\n{}\n".format(test_cov.shape, test_cov))

# Mittelwert pro Merkmalsspalte
x_mean = np.mean(X, axis = 0).reshape(1, -1)
print("Mittelwertvektor: {}\nmit Shape: {}\n".format(x_mean, x_mean.shape))
n = len(X)
temp = np.subtract(X, x_mean)
cov_mat = 1/(n-1)*((temp.T).dot(temp))
print("Kovarianzmatrix: {}\n mit Shape: {}".format(cov_mat, cov_mat.shape))

Test: 
(4, 4)
[[ 0.68569351 -0.03926846  1.27368233  0.5169038 ]
 [-0.03926846  0.18800403 -0.32171275 -0.11798121]
 [ 1.27368233 -0.32171275  3.11317942  1.29638747]
 [ 0.5169038  -0.11798121  1.29638747  0.58241432]]

Mittelwertvektor: [[5.84333333 3.054      3.75866667 1.19866667]]
mit Shape: (1, 4)

Kovarianzmatrix: [[ 0.68569351 -0.03926846  1.27368233  0.5169038 ]
 [-0.03926846  0.18800403 -0.32171275 -0.11798121]
 [ 1.27368233 -0.32171275  3.11317942  1.29638747]
 [ 0.5169038  -0.11798121  1.29638747  0.58241432]]
 mit Shape: (4, 4)


# 2 - Eigendekomposition - Berechnung Eigenvektoren and Eigenwerte

Im folgenden wird das in der Vorlesung (Prof. Laubenheimer) besprochene Vorgehen Schritt-für-Schritt umgesetzt, mit einer kleinen Abwandlung. Die einzelnen Berechnungsschritte sind für eine eigenständige Implementierung Ihrerseits im Rahmen dieses Labors zu aufwendig. Daher können Sie auf Implementierungen von Bibliotheks-Funktionen zurückgreifen. Welche diese konkret sind wird jedoch nicht vorweggenommen. Diese sind Ihrerseits anhand von Recherche-Arbeiten herauszufinden.

## 2.1. Charakteristisches Polynom
Der erste Schritt ist die Aufstellung des charakteristischen Polynoms. 

In [7]:
# TODO: implement
#bezieht sich auf C
cp = np.poly(cov_mat)
print(cp)

[ 1.00000000e+00 -4.56929128e+00  1.48186720e+00 -1.12910083e-01
  1.90327580e-03]


## 2.2. Eigenwerte (Nullstellen)
Der zweite Schritt ist die Berechnung der Nullstellen anhand des charakteristischen Polynoms. Damit einhergehend sind die Eigenwerte bestimmt.

In [8]:
# TODO: implement

#Calculate the roots of p using roots. The roots of the characteristic polynomial are the eigenvalues of matrix A.
eig_vals = np.roots(cp)
print(eig_vals)

eigvals,eigvecs = la.eig(cov_mat)
print("TEST: {}".format(eigvals))

[4.22484077 0.24224357 0.07852391 0.02368303]
TEST: [4.22484077+0.j 0.24224357+0.j 0.07852391+0.j 0.02368303+0.j]


## 2.3. Eigenvektoren
Der dritte Schritte ist die Berechnung der Eigenvektoren durch Lösung der Gleichung: <br>
$C−\lambda_i \cdot I \cdot \vec{e}_i = \vec{0}$.

In der Vorlesung und in der Klausur (sofern diese Aufgabe gestellt wird), wird an dieser Stelle das Lineare Gleichungssystem (LGS) gelöst. Dies sollten Sie an dieser Stelle auch tun! Leider erzeugen jedoch die gängigen LGS-Solver, wie bspw. <b>scipy.linalg.solve()</b> und <b>numpy.linalg.solve()</b>, lediglich die triviale Lösung. Gerne können Sie das erproben.<br>

<b>Achtung!</b>: Hier kommt der Teil, an dem wir vom Skript abweichen: <br>
* Stellen Sie für jede Nullstelle <i>i</i> (Eigenwert) folgende Matrix m auf: $m_i = (C−\lambda_i \cdot I).$
* Führen Sie für jede Nullstelle auf Basis der Matrizen eine QR-Zerlegung (https://de.wikipedia.org/wiki/QR-Zerlegung) durch. Dh finden Sie eine Funktion die dies berechnet. Die QR-Zerlegung unterstützt Sie bei der Lösung eines linearen Gleichungssystems. Das Resultat der Zerlegung sind die zwei Matrizen <b>q</b> und <b>r</b>.<br>
* Bestimmen Sie den <i>Rang der Matrix</i> wiefolgt: Ermitteln Sie in <b>r</b> die Zeile, in der sämtliche Werte 0 sind bzw. "nah dran". Der Rang der Matrix ist der Wert der ermittelten Zeile minus 1. <br> Zum Beispiel: Angenommen, in der Zeile 4 sind alle Werte nah an 0, dann ist der Rang der Matrix: Zeile 4 minus 1 = 3. (Grafik: Rot) <br> 
* Bestimmen Sie die <i>Eigenvektoren</i> wiefolgt: Ermitteln Sie in <b>q</b> die vom Ende betrachtet n-r Spalten (mit n=Dimension, r=Rang). In diesen Spalten sind die Eigenvektoren enthalten. <br> Zum Beispiel: Angenommen die Dimension n=4 und der Rang r=3, dann sind die Eigenvektoren in der vom Ende betrachteten Spalte n-r= 4-3= 1, dh die erste Spalte vom Ende betrachtet, bzw. die vierte Spalte vom Anfang betrachtet. (Grafik: Grün)

<img src="./Figures/q_r_color.png" alt="drawing" style="width:400px;"/>

Nochmals der Hinweis führen Sie dieses Verfahren für alle Nullstellen durch und erzeugen Sie eine geeignete Datenstruktur.<br>

Testen Sie das Ergebnis Ihrer Implementierung anhand der Funktion <b>numpy.linalg.eig()</b>.

In [9]:
# TODO: implement
# Es gilt:
# q * q = I 
# q.T * q = I
# r ist eine obere Dreiecksmatirx

eig_vecs = []
I = np.array([[1,0,0,0], [0,1,0,0],[0,0,1,0],[0,0,0,1]]).reshape(4, -1)
for e in eig_vals:
    m_ = np.subtract(cov_mat, (e*I))

    q, r = np.linalg.qr(m_, "complete")
    r_sum = np.sum(r, axis=1) # sum the rows and get the minimum from absulute val
    min_sum = min(abs(r_sum))
    rank = 0
    for i, val in enumerate(r_sum):
        if r_sum[abs(r_sum) == min_sum]:
            rank = i
    q_line = q.shape[1] - rank

    print("Rank: {}, Line to look for in q: -{}".format(rank, q_line))
    print(q[:,-q_line])
    eig_vecs.append(q[:,-q_line])

    
eig_vecs = np.array(eig_vecs)
print("Calculated Eigenvectors: \n{}\n".format(eig_vecs.T))
print("Test:\n{}".format(np.linalg.eig(cov_mat)[1]))

Rank: 3, Line to look for in q: -1
[ 0.36158968 -0.08226889  0.85657211  0.35884393]
Rank: 3, Line to look for in q: -1
[-0.65653988 -0.72971237  0.1757674   0.07470647]
Rank: 3, Line to look for in q: -1
[-0.58099728  0.59641809  0.07252408  0.54906091]
Rank: 3, Line to look for in q: -1
[ 0.31725455 -0.32409435 -0.47971899  0.75112056]
Calculated Eigenvectors: 
[[ 0.36158968 -0.65653988 -0.58099728  0.31725455]
 [-0.08226889 -0.72971237  0.59641809 -0.32409435]
 [ 0.85657211  0.1757674   0.07252408 -0.47971899]
 [ 0.35884393  0.07470647  0.54906091  0.75112056]]

Test:
[[ 0.36158968 -0.65653988 -0.58099728  0.31725455]
 [-0.08226889 -0.72971237  0.59641809 -0.32409435]
 [ 0.85657211  0.1757674   0.07252408 -0.47971899]
 [ 0.35884393  0.07470647  0.54906091  0.75112056]]


# 3 - Auswahl der Hauptkomponenten (Principal Components)

## Sortierung der Eigenpaare (Eigenpairs)

Das Ziel der PCA ist es, die Dimensionalität des ursprünglichen Merkmalsraums zu reduzieren, indem dieser auf einen kleineren Unterraum projiziert wird, in dem die Eigenvektoren die Achsen bilden. Die Eigenvektoren definieren jedoch nur die Richtungen der neuen Achse, da sie alle die gleiche Einheitslänge 1 haben. Dies kann durch die folgenden beiden Codezeilen bestätigt werden:

In [10]:
for ev in eig_vecs.T:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Ok Ok!')

Ok Ok!


Um zu entscheiden, welche/r Eigenvektor/en entfallen kann/können, ohne dass zu viele Informationen für die Konstruktion des niederdimensionalen Unterraums verloren gehen, müssen die entsprechenden Eigenwerte untersucht werden: Die Eigenvektoren mit den niedrigsten Eigenwerten enthalten die geringste Information über die Verteilung der Daten; diese können entfallen. Zu diesem Zweck werden die Eigenwerte von den höchsten zu den niedrigsten geordnet/ sortiert, um die besten k Eigenvektoren auszuwählen.

Erzeugen Sie eine Liste von Tupeln, welche die Eigenwerte und Eigenvektoren enthält. Und sortieren Sie diese entsprechend.

In [72]:
# Make a list of (eigenvalue, eigenvector) tuples
#Die resultierenden Eigenvektoren 𝑒𝑖 sind, absteigend sortiert nach dem Wert der zugehörigen 
#Eigenwerte 𝜆𝑖, die Hauptkomponenten der Ursprungsdaten.
# TODO: implement
eig_pairs = []
for i in range(len(eig_vals)):
     eig_pairs.append((eig_vals[i], eig_vecs[i]))

eig_pairs.sort(reverse = True, key= lambda x: x[0])
print(eig_pairs)

[(4.224840768320106, array([ 0.36158968, -0.08226889,  0.85657211,  0.35884393])), (0.24224357162751514, array([-0.65653988, -0.72971237,  0.1757674 ,  0.07470647])), (0.07852390809415496, array([-0.58099728,  0.59641809,  0.07252408,  0.54906091])), (0.023683027126002124, array([ 0.31725455, -0.32409435, -0.47971899,  0.75112056]))]


## Explained Variance

Nach der Sortierung der Eigenpaare stellen Sie sich möglicherweise folgende Frage: <i>"Wie viele Hauptkomponenten werden für den neuen Feature-Unterraum ausgewählt?"</i> <br>

Ein nützliches Maß ist die sogenannte "explained variance", die sich aus den Eigenwerten berechnen lässt. Die "explained variance" gibt an, wie viel Informationen (Varianz) den einzelnen Hauptkomponenten zugeordnet werden können.

Ermitteln Sie die "explained variance", indem sie den prozentualen Anteil der einzelnen Eigenwerten aus der Gesamtheit Eigenwerten berechnen.

In [73]:
sum_eig_val = np.sum(eig_vals)
explainedVariance = []
for ew, ev in eig_pairs:
    explainedVariance.append((ew/sum_eig_val))
print(explainedVariance)

[0.9246162071742681, 0.053015567850535045, 0.0171851395250069, 0.005183085450189979]


Bewerten Sie die "explained variance".

## Projektionsmatrix W

Es folgt die Konstruktion der Projektionsmatrix, mit der die Iris-Daten in den neuen Feature-Unterraum transformiert werden. Die Projektionsmatrix ist im Grunde genommen eine Matrix der konkatenierten Top-k-Eigenvektoren. <br>

Es erfolgt eine Reduktion des 4-dimensionalen Merkmalsraum auf einen 2-dimensionalen Merkmalsunterraum, indem die "Top 2"-Eigenvektoren mit den höchsten Eigenwerten ausgewählt werden, um die d$\times$k-dimensionale Projektionsmatrix (auch: Eigenvektormatrix) <b>W</b> zu konstruieren. 

Erzeugen Sie die Projektionsmatrix <b>W</b> (mit einem shape: (4, 2)) aus den Eigenpairs.

In [74]:
# TODO: implement
matrix_w = np.concatenate(([eig_pairs[0][1]], [eig_pairs[1][1]]), axis=0).T
matrix_w, matrix_w.shape

(array([[ 0.36158968, -0.65653988],
        [-0.08226889, -0.72971237],
        [ 0.85657211,  0.1757674 ],
        [ 0.35884393,  0.07470647]]),
 (4, 2))

# 4 - Projektion in den neuen Feature-Unterraum

In diesem letzten Schritt wird die 4$\times$2-dimensionale Projektionsmatrix W verwendet, um die Daten über die Gleichung auf den neuen Unterraum zu transformieren:

$Y=X \cdot W$ <br>

wobei Y eine 150$\times$2-Matrix der transformierten Daten ist.

In [81]:
# TODO: implement
Y = np.dot(X, matrix_w)
Y = np.concatenate((Y,y), axis=1)
print(Y[:2])

[[2.8271359726790295 -5.641331045573342 'Iris-setosa']
 [2.7959524821488464 -5.145166883252927 'Iris-setosa']]


Führen Sie folgenden Code aus und visualisieren Sie das Ergebnis.

In [82]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(6, 4))
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'red', 'green')):
        plt.scatter(Y[Y[:, 2]==lab, 0],#row condition, column index
                    Y[Y[:, 2]==lab, 1],
                    label=lab,
                    c=col)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend(loc='lower center')
    plt.tight_layout()
    plt.show()


Das Ergebnis sollte wiefolgt aussehen: 
<img src="./Figures/pca_result.png" alt="drawing" style="width:400px;"/>